# TASK-1

## Q1

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%config InlineBackend.figure_format='retina'
Laying_df = pd.read_csv('.\HAR\Combined\Train\LAYING\Subject_1.csv')
Sitting_df = pd.read_csv('.\HAR\Combined\Train\SITTING\Subject_1.csv')
Standing_df = pd.read_csv('.\HAR\Combined\Train\STANDING\Subject_1.csv')
Walking_df = pd.read_csv('.\HAR\Combined\Train\WALKING\Subject_1.csv')
WalkingDown_df = pd.read_csv('.\HAR\Combined\Train\WALKING_DOWNSTAIRS\Subject_1.csv')
WalkingUp_df = pd.read_csv('.\HAR\Combined\Train\WALKING_UPSTAIRS\Subject_1.csv')
# List of DataFrames
dfs = [Laying_df,Sitting_df,Standing_df,Walking_df,WalkingDown_df,WalkingUp_df]
colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan']
labels = ['LAYING','SITTING','STANDING','WALKING','WALKING_DOWN','WALKING_UP']
for j, axis in enumerate(['accx', 'accy', 'accz']):
    fig, ax = plt.subplots(nrows=6, figsize=(10, 10))
    for i, df in enumerate(dfs):
        ax[i].plot(df[axis], label=f'{labels[i]}', lw=0.75,color = colors[i])
        ax[i].legend(loc='upper right',fontsize='small') 
    fig.text(0.04, 0.5, f'{axis}', va='center', ha='center', rotation='vertical', fontsize=12)
    fig.text(0.5, 0.04, 'Time (samples)', va='center', ha='center', fontsize=12)
    fig.text(0.5, 0.9, f'{axis} vs Time (WaveForm)', va='center', ha='center', fontsize=12)
    plt.show()
    

Yes, It is possible for the model to classify models based on the data.

As you can see in the above plots, each action has a different pattern in the $acc_x,acc_y,acc_z$.
Though depending on our model parameters and settings, it might be a little difficult for the model to differentiate betweens labels like laying and sitting, standing and Walking and WalkingUp. 

## Q2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


dfs = [np.sqrt(Laying_df['accx']**2+Laying_df['accy']**2+Laying_df['accz']**2),
       np.sqrt(Sitting_df['accx']**2+Sitting_df['accy']**2+Sitting_df['accz']**2)
       ,np.sqrt(Standing_df['accx']**2+Standing_df['accy']**2+Standing_df['accz']**2)
       ,np.sqrt(Walking_df['accx']**2+Walking_df['accy']**2+Walking_df['accz']**2)
       ,np.sqrt(WalkingDown_df['accx']**2+WalkingDown_df['accy']**2+WalkingDown_df['accz']**2)
       ,np.sqrt(WalkingUp_df['accx']**2+WalkingUp_df['accy']**2+WalkingUp_df['accz']**2)]

labels = ['LAYING','SITTING','STANDING','WALKING','WALKING_DOWN','WALKING_UP']

fig, ax = plt.subplots(nrows=6, figsize=(10, 15))
colors = ['blue', 'green', 'red', 'purple', 'orange', 'cyan']

for i, df in enumerate(dfs):
    ax[i].plot(df, label=f'{labels[i]}', lw=0.75,color = colors[i])
    ax[i].legend(loc='upper right',fontsize='small') 

# Shared y-label
fig.text(0.04, 0.5, '$(acc_x^2+acc_y^2+acc_z^2)$', va='center', ha='center', rotation='vertical', fontsize=12)
# Shared x-label
fig.text(0.5, 0.04, 'Time (samples)', va='center', ha='center', fontsize=12)
plt.show()

As you can see from the above data, it is quite possible to distinguish between between static and dynamic without a ML model. As long as the you set a variable that detects the fluctuation in the linear accelaration, it will be very clear whether it is static or dynamic. But to futher detect between the subclasses of static and dynamic can only be done with the help of ML models